# Topic Modeling with gensim
We'll try out [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) in [gensim](http://radimrehurek.com/gensim/index.html) on the [20 Newsgroups dataset](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) with some simple preprocessing.

#### Install gensim

In [1]:
# pip install --upgrade gensim

##### imports

In [2]:
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Let's retain only a subset of the 20 categories in the original 20 Newsgroups Dataset.

In [160]:
# Set categories
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball']#, 
              #'rec.motorcycles', 'sci.space', 'talk.politics.mideast']
# Download the training subset of the 20 NG dataset, with headers, footers, quotes removed
# Only keep docs from the 6 categories above
ng_train = datasets.fetch_20newsgroups(subset='train', categories=categories, 
                                      remove=('headers', 'footers', 'quotes'))

In [161]:
# Take a look at the first doc
ng_train.data[0]

u'\n\n\nI happen to be a big fan of Jayson Stark.  He is a baseball writer for the \nPhiladelphia Inquirer.  Every tuesday he writes a "Week in Review" column.  \nHe writes about unusual situations that occured during the week.  Unusual\nstats.  He has a section called "Kinerisms of the Week" which are stupid\nlines by Mets brodcaster Ralph Kiner.  Every year he has the LGTGAH contest.\nThat stands for "Last guy to get a hit."  He also writes for Baseball \nAmerica.  That column is sort of a highlights of "Week in Review."  If you \ncan, check his column out sometime.  He might make you laugh.\n\nRob Koffler\n'

## Document Preprocessing
We'll need to generate a term-document matrix of word (token) counts for use in LDA.

We'll use `sklearn`'s `CountVectorizer` to generate our term-document matrix of counts. We'll make use of a few parameters to accomplish the following preprocessing of the text documents all within the `CountVectorizer`:
* `analyzer=word`: Tokenize by word
* `ngram_range=(1,2)`: Keep all 1 and 2-word grams
* `stop_words=english`: Remove all English stop words
* `token_pattern=\\b[a-z][a-z]+\\b`: Match all tokens with 2 or more (strictly) alphabet characters
* `min_df=2`: Words must appear in at least 2 documents
* `max_df=0.02`: Words must appear in less than 2% of the documents

In [162]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(analyzer='word',
                                  ngram_range=(1, 2), stop_words='english',
                                  token_pattern='\\b[a-z][a-z]+\\b', max_df=0.02, min_df=2)
count_vectorizer.fit(ng_train.data)

CountVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.02, max_features=None, min_df=2,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [163]:
# Create the term-document matrix
# Transpose it so the terms are the rows
ng_vecs = count_vectorizer.transform(ng_train.data).transpose()
ng_vecs.shape

(15077, 1661)

##### Convert to gensim
We need to convert our sparse `scipy` matrix to a `gensim`-friendly object called a Corpus:

In [164]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(ng_vecs)

##### Map matrix rows to words (tokens)
We need to save a mapping (dict) of row id to word (token) for later use by gensim:

In [165]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.iteritems())

In [166]:
len(id2word)

15077

## LDA
At this point we can simply plow ahead in creating an LDA model.  It requires our corpus of word counts, mapping of row ids to words, and the number of topics (3).

In [176]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus, id2word=id2word, num_topics=3, passes=10)

Let's take a look at what happened.  Here are the 5 most important words for each of the 3 topics we found:

In [168]:
lda.print_topics(num_words=5, num_topics=3)

[(0,
  u'0.004*jesus + 0.002*matthew + 0.002*fallacy + 0.001*conclusion + 0.001*christian'),
 (1, u'0.006*jpeg + 0.003*pub + 0.002*display + 0.002*faq + 0.002*processing'),
 (2,
  u'0.002*morris + 0.002*alomar + 0.001*career + 0.001*double + 0.001*clemens')]

#### Topic Space
If we want to map our documents to the topic space we need to actually use the LdaModel transformer that we created above, like so:

In [169]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]

In [170]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

Now we can take a look at the document vectors in the topic space, which are measures of the component of each document along each topic.  Thus, at most a document vector can have num_topics=3 nonzero components in the topic space, and most have far fewer.

In [174]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:5]

[[(0, 0.14074902976825285),
  (1, 0.84829616315399592),
  (2, 0.010954807077751436)],
 [(0, 0.030234919130326277),
  (1, 0.031789655630704947),
  (2, 0.93797542523896871)],
 [(0, 0.76029455927147815),
  (1, 0.22872091684900533),
  (2, 0.010984523879516397)],
 [(0, 0.82219006971303565),
  (1, 0.091629552860028857),
  (2, 0.086180377426935559)],
 [(0, 0.017264908074076341),
  (1, 0.017108573144132416),
  (2, 0.96562651878179118)]]

In [175]:
ng_train.data[0:5]

[u'\n\n\nI happen to be a big fan of Jayson Stark.  He is a baseball writer for the \nPhiladelphia Inquirer.  Every tuesday he writes a "Week in Review" column.  \nHe writes about unusual situations that occured during the week.  Unusual\nstats.  He has a section called "Kinerisms of the Week" which are stupid\nlines by Mets brodcaster Ralph Kiner.  Every year he has the LGTGAH contest.\nThat stands for "Last guy to get a hit."  He also writes for Baseball \nAmerica.  That column is sort of a highlights of "Week in Review."  If you \ncan, check his column out sometime.  He might make you laugh.\n\nRob Koffler\n',
 u'\nHere\'s one I remember: (sort of)\nYogi\'s asleep in a hotel room late at night and gets a call from someone.\nAfter he answers the phone the person at the other end asks if he woke Yogi\nup. Yogi answered, "No, the phone did."',
 u'\n\n\tSorry, I was, but I somehow have misplaced my diskette from the last \ncouple of months or so. However, thanks to the efforts of Bobby,

## On your own...
- Go get some of the NIPS papers from [here](https://archive.ics.uci.edu/ml/datasets/Bag+of+Words).  
- Try performing LDA on this data with gensim
- Play with some of the preprocessing options and parameters for LDA, observe what happens
- See if you can use the resulting topic space to extract topic vectors and cluster some documents
- How do your results look?